# Merging Census and Health Data
Health data is saved as "health.csv" so change that if you have it labeled as something else.

To change variables extracted in census data, change codes under the "variables of interest section" and update the change name section.

To change variables extracted in census data, change "columns to keep" section.

In [1]:
!pip install censusdata
import pandas as pd
import censusdata

Defaulting to user installation because normal site-packages is not writeable


/Users/janak/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Census Data

In [ ]:
# --- Variables of interest ---
variables = [
    'B19013_001E',  # Median income
    'B01002_001E',  # Median age
    'B17001_002E',  # Poverty
    'B17001_001E',  # Poverty population
    'B15003_017E',  # HS grad
    'B15003_022E',  # Bachelor's degree
    'B15003_001E',  # Education population
    'B23025_005E',  # Unemployed
    'B23025_001E',   # Unemployed population
        # --- RACE (B02001) ---
    'B02001_001E',  # Total
    'B02001_002E',  # White alone
    'B02001_003E',  # Black or African American alone
    'B02001_004E',  # American Indian/Alaska Native alone
    'B02001_005E',  # Asian alone
    'B02001_006E',  # NH/Other Pacific Islander alone
    'B02001_007E',  # Some other race alone
    'B02001_008E',  # Two or more races
    'B02001_009E',  # Two races incl. Some other race
    'B02001_010E',  # Two races excl. Some other race, and 3+ races
        # Hispanic 
    'B03002_001E',  # Hispanic Total
    'B03002_012E'   # Hispanic or Latino
]


# --- Valid state FIPS codes (50 states + DC) ---
valid_states = [
    '01','02','04','05','06','08','09','10','11','12','13','15','16','17','18','19',
    '20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35',
    '36','37','38','39','40','41','42','44','45','46','47','48','49','50','51','53',
    '54','55','56'
]

# --- Function to parse censusgeo objects ---
def split_geo(cgeo):
    codes = dict(cgeo.geo)
    name_parts = cgeo.name.split(", ")
    tract_name = name_parts[0]
    county_name = name_parts[1]
    state_name = name_parts[2]
    return pd.Series([
        codes.get('state'), codes.get('county'), codes.get('tract'),
        state_name, county_name, tract_name
    ])

# --- Download and process data for all states ---
all_data = []

for state in valid_states:
    print(f"Downloading data for state FIPS: {state}")
    data = censusdata.download(
        'acs5', 2015,
        censusdata.censusgeo([('state', state), ('county', '*'), ('tract', '*')]),
        variables
    )
    
    # Extract geo info
    geo_cols = data.index.to_series().apply(split_geo)
    geo_cols.columns = [
        'State_FIPS', 'County_FIPS', 'Tract_FIPS',
        'State_Name', 'County_Name', 'Tract_Name'
    ]
    
    # Combine geo info with data
    data_clean = pd.concat([geo_cols, data], axis=1).reset_index(drop=True)
    all_data.append(data_clean)

# --- Combine all states ---
census_df = pd.concat(all_data, ignore_index=True)

# --- Create 11-digit tract FIPS ---
census_df["Tract_FIPS_full"] = (
    census_df["State_FIPS"].astype(str).str.zfill(2) +
    census_df["County_FIPS"].astype(str).str.zfill(3) +
    census_df["Tract_FIPS"].astype(str).str.zfill(6)
)

#rename variables
census_df = census_df.rename(columns={
    'B19013_001E': 'Median_Income',
    'B01002_001E': 'Median_Age',
    'B17001_002E': 'Poverty_Count',
    'B17001_001E': 'Poverty_Pop',
    'B15003_017E': 'HS_Grad_Count',
    'B15003_022E': 'Bachelors_Count',
    'B15003_001E': 'Education_Pop',
    'B23025_005E': 'Unemployed_Count',
    'B23025_001E': 'Unemployed_Pop',
    'B02001_001E': 'Race_Total',
    'B02001_002E': 'White_Alone',
    'B02001_003E': 'Black_Alone',
    'B02001_004E': 'AIAN_Alone',
    'B02001_005E': 'Asian_Alone',
    'B02001_006E': 'NHPI_Alone',
    'B02001_007E': 'Other_Alone',
    'B02001_008E': 'TwoPlus',
    'B02001_009E': 'Two_incl_Other',
    'B02001_010E': 'Two_excl_Other_or_3plus',
    'B03002_001E': 'Hispanic_Total',
    'B03002_012E': 'Hispanic_Any'
})

import numpy as np

race_num_cols = [
    'White_Alone','Black_Alone','AIAN_Alone','Asian_Alone',
    'NHPI_Alone','Other_Alone','TwoPlus'
]

# Convert all race counts and totals to numeric
for c in race_num_cols:
    census_df[c] = pd.to_numeric(census_df[c], errors='coerce')

race_total = pd.to_numeric(census_df['Race_Total'], errors='coerce')

# Replace 0 totals with NaN to avoid divide-by-zero
den = race_total.replace(0, np.nan)

# Compute percentages
for c in race_num_cols:
    census_df[f'Pct_{c}'] = 100.0 * census_df[c] / den

# (Optional) A single “majority race” label per tract
race_pct_cols = [
    'Pct_White_Alone','Pct_Black_Alone','Pct_AIAN_Alone','Pct_Asian_Alone','Pct_NHPI_Alone','Pct_Other_Alone','Pct_TwoPlus'
]
# Rename columns above already match this list; if not, adjust names accordingly.

census_df['Majority_Race'] = census_df[race_pct_cols].idxmax(axis=1).str.replace('Pct_','').str.replace('_Alone','')

print("Race variables added. Example rows:")
print(census_df[['Tract_FIPS_full','Race_Total'] + race_pct_cols].head())


print("ACS tract-level data ready with full 11-digit FIPS!")


Race variables added. Example rows:
  Tract_FIPS_full  Race_Total  Pct_White_Alone  Pct_Black_Alone  \
0     01103005109        4792        64.232053        28.338898   
1     01103005106        5739        80.989719        13.922286   
2     01103005107        4883        92.197420         2.170797   
3     01103005108        3787        76.894640        18.061790   
4     01103005701        2784        97.557471         0.000000   

   Pct_AIAN_Alone  Pct_Asian_Alone  Pct_NHPI_Alone  Pct_Other_Alone  \
0        3.025876         0.000000             0.0         0.000000   
1        0.000000         1.254574             0.0         0.000000   
2        0.000000         2.396068             0.0         0.163834   
3        0.396092         0.000000             0.0         0.000000   
4        1.400862         0.000000             0.0         0.000000   

   Pct_TwoPlus  
0     4.403172  
1     3.833420  
2     3.071882  
3     4.647478  
4     1.041667  
ACS tract-level data ready with 

/var/folders/kt/_bzbz3vd5qbcfjnt5vssmqxh0000gn/T/ipykernel_59855/2095683370.py:133: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  census_df['Majority_Race'] = census_df[race_pct_cols].idxmax(axis=1).str.replace('Pct_','').str.replace('_Alone','')


In [4]:
# --- % Hispanic (from B03002 you already pulled) ---
census_df['Hispanic_Total'] = pd.to_numeric(census_df['Hispanic_Total'], errors='coerce')
census_df['Hispanic_Any']   = pd.to_numeric(census_df['Hispanic_Any'], errors='coerce')
den_h = census_df['Hispanic_Total'].replace(0, np.nan)
census_df['Pct_Hispanic'] = 100.0 * census_df['Hispanic_Any'] / den_h

# --- Build "Other" share and 5-bucket majority label ---
# ensure all % columns are numeric
for c in ['Pct_White_Alone','Pct_Black_Alone','Pct_Asian_Alone',
          'Pct_AIAN_Alone','Pct_NHPI_Alone','Pct_Other_Alone','Pct_TwoPlus','Pct_Hispanic']:
    census_df[c] = pd.to_numeric(census_df[c], errors='coerce')

# Other = AIAN + NHPI + Other_Alone + TwoPlus
census_df['Pct_Other'] = (
    census_df[['Pct_AIAN_Alone','Pct_NHPI_Alone','Pct_Other_Alone','Pct_TwoPlus']]
    .fillna(0).sum(axis=1)
)

# pick the max across White, Black, Asian, Hispanic, Other
share_table = pd.DataFrame({
    'White':    census_df['Pct_White_Alone'],
    'Black':    census_df['Pct_Black_Alone'],
    'Asian':    census_df['Pct_Asian_Alone'],
    'Hispanic': census_df['Pct_Hispanic'],
    'Other':    census_df['Pct_Other'],
})
census_df['Majority_Race_Clean'] = share_table.idxmax(axis=1)

# sanity check
print(census_df['Majority_Race_Clean'].value_counts(dropna=False).head())


Majority_Race_Clean
White       59371
Black        7047
Hispanic     4581
Asian        1066
Other         991
Name: count, dtype: int64


In [5]:
census_df.head()

,State_FIPS,County_FIPS,Tract_FIPS,State_Name,County_Name,Tract_Name,Median_Income,Median_Age,Poverty_Count,Poverty_Pop,...,Pct_Black_Alone,Pct_AIAN_Alone,Pct_Asian_Alone,Pct_NHPI_Alone,Pct_Other_Alone,Pct_TwoPlus,Majority_Race,Pct_Hispanic,Majority_Race_Clean,Pct_Other
0,01,103,005109,Alabama,Morgan County,Census Tract 51.09,29644.0,29.6,1476,4792,...,28.338898,3.025876,0.000000,0.0,0.000000,4.403172,White,20.993322,White,7.429048
1,01,103,005106,Alabama,Morgan County,Census Tract 51.06,35864.0,43.3,1186,5723,...,13.922286,0.000000,1.254574,0.0,0.000000,3.833420,White,1.045478,White,3.833420
2,01,103,005107,Alabama,Morgan County,Census Tract 51.07,66739.0,43.5,137,4853,...,2.170797,0.000000,2.396068,0.0,0.163834,3.071882,White,1.945525,White,3.235716
3,01,103,005108,Alabama,Morgan County,Census Tract 51.08,64632.0,45.8,566,3787,...,18.061790,0.396092,0.000000,0.0,0.000000,4.647478,White,3.221547,White,5.043570
4,01,103,005701,Alabama,Morgan County,Census Tract 57.01,46306.0,38.4,571,2784,...,0.000000,1.400862,0.000000,0.0,0.000000,1.041667,White,0.000000,White,2.442529


In [6]:
census_df.columns

Index(['State_FIPS', 'County_FIPS', 'Tract_FIPS', 'State_Name', 'County_Name',
       'Tract_Name', 'Median_Income', 'Median_Age', 'Poverty_Count',
       'Poverty_Pop', 'HS_Grad_Count', 'Bachelors_Count', 'Education_Pop',
       'Unemployed_Count', 'Unemployed_Pop', 'Race_Total', 'White_Alone',
       'Black_Alone', 'AIAN_Alone', 'Asian_Alone', 'NHPI_Alone', 'Other_Alone',
       'TwoPlus', 'Two_incl_Other', 'Two_excl_Other_or_3plus',
       'Hispanic_Total', 'Hispanic_Any', 'Tract_FIPS_full', 'Pct_White_Alone',
       'Pct_Black_Alone', 'Pct_AIAN_Alone', 'Pct_Asian_Alone',
       'Pct_NHPI_Alone', 'Pct_Other_Alone', 'Pct_TwoPlus', 'Majority_Race',
       'Pct_Hispanic', 'Majority_Race_Clean', 'Pct_Other'],
      dtype='object')

## Health Data

In [7]:
health_df = pd.read_csv("health.csv")

In [8]:
health_df.head()

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Population2010,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,...,PAPTEST_Crude95CI,PHLTH_CrudePrev,PHLTH_Crude95CI,SLEEP_CrudePrev,SLEEP_Crude95CI,STROKE_CrudePrev,STROKE_Crude95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,Geolocation
0,CA,Los Angeles,644000,6037206032,0644000-06037206032,5275,28.0,"(22.7, 33.2)",19.9,"(18.8, 20.9)",...,"(81.5, 86.0)",17.3,"(15.3, 19.2)",38.9,"(37.2, 40.4)",4.0,"( 3.5, 4.5)",18.8,"(12.6, 25.6)",POINT (-118.224698433 34.0470512474)
1,CA,Pasadena,656000,6037462001,0656000-06037462001,3974,23.3,"(19.2, 27.8)",16.7,"(16.0, 17.5)",...,"(84.1, 87.5)",15.3,"(13.9, 16.8)",40.2,"(38.8, 41.5)",3.2,"( 2.9, 3.5)",18.1,"(13.7, 22.8)",POINT (-118.143832177 34.1633689905)
2,CA,Fullerton,628000,6059011504,0628000-06059011504,5473,11.7,"( 9.6, 14.5)",8.3,"( 7.9, 8.7)",...,"(75.4, 80.9)",8.5,"( 7.7, 9.5)",33.4,"(31.6, 35.1)",1.2,"( 1.1, 1.3)",11.9,"( 8.8, 16.0)",POINT (-117.883112998 33.8820402343)
3,TX,Longview,4843888,48183000401,4843888-48183000401,3371,19.3,"(16.8, 21.9)",21.1,"(20.2, 21.9)",...,"(76.5, 81.9)",11.8,"(10.8, 12.8)",34.9,"(33.4, 36.3)",2.7,"( 2.5, 3.0)",15.1,"(10.8, 20.4)",POINT (-94.7523262965 32.5544549842)
4,WI,Madison,5548000,55025002200,5548000-55025002200,4254,10.5,"( 8.7, 12.6)",23.3,"(21.9, 24.6)",...,"(83.4, 88.0)",11.3,"( 9.8, 12.7)",30.8,"(29.3, 32.2)",2.9,"( 2.5, 3.3)",12.9,"( 7.6, 19.8)",POINT (-89.3623704161 43.1211036947)


In [9]:
#extract the 11-digit tract FIPS after the dash
health_df["Tract_FIPS_full"] = health_df["Place_TractID"].str.split("-").str[-1]

In [10]:
health_df.head()

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Population2010,ACCESS2_CrudePrev,ACCESS2_Crude95CI,ARTHRITIS_CrudePrev,ARTHRITIS_Crude95CI,...,PHLTH_CrudePrev,PHLTH_Crude95CI,SLEEP_CrudePrev,SLEEP_Crude95CI,STROKE_CrudePrev,STROKE_Crude95CI,TEETHLOST_CrudePrev,TEETHLOST_Crude95CI,Geolocation,Tract_FIPS_full
0,CA,Los Angeles,644000,6037206032,0644000-06037206032,5275,28.0,"(22.7, 33.2)",19.9,"(18.8, 20.9)",...,17.3,"(15.3, 19.2)",38.9,"(37.2, 40.4)",4.0,"( 3.5, 4.5)",18.8,"(12.6, 25.6)",POINT (-118.224698433 34.0470512474),06037206032
1,CA,Pasadena,656000,6037462001,0656000-06037462001,3974,23.3,"(19.2, 27.8)",16.7,"(16.0, 17.5)",...,15.3,"(13.9, 16.8)",40.2,"(38.8, 41.5)",3.2,"( 2.9, 3.5)",18.1,"(13.7, 22.8)",POINT (-118.143832177 34.1633689905),06037462001
2,CA,Fullerton,628000,6059011504,0628000-06059011504,5473,11.7,"( 9.6, 14.5)",8.3,"( 7.9, 8.7)",...,8.5,"( 7.7, 9.5)",33.4,"(31.6, 35.1)",1.2,"( 1.1, 1.3)",11.9,"( 8.8, 16.0)",POINT (-117.883112998 33.8820402343),06059011504
3,TX,Longview,4843888,48183000401,4843888-48183000401,3371,19.3,"(16.8, 21.9)",21.1,"(20.2, 21.9)",...,11.8,"(10.8, 12.8)",34.9,"(33.4, 36.3)",2.7,"( 2.5, 3.0)",15.1,"(10.8, 20.4)",POINT (-94.7523262965 32.5544549842),48183000401
4,WI,Madison,5548000,55025002200,5548000-55025002200,4254,10.5,"( 8.7, 12.6)",23.3,"(21.9, 24.6)",...,11.3,"( 9.8, 12.7)",30.8,"(29.3, 32.2)",2.9,"( 2.5, 3.3)",12.9,"( 7.6, 19.8)",POINT (-89.3623704161 43.1211036947),55025002200


In [11]:
health_df.columns

Index(['StateAbbr', 'PlaceName', 'PlaceFIPS', 'TractFIPS', 'Place_TractID',
       'Population2010', 'ACCESS2_CrudePrev', 'ACCESS2_Crude95CI',
       'ARTHRITIS_CrudePrev', 'ARTHRITIS_Crude95CI', 'BINGE_CrudePrev',
       'BINGE_Crude95CI', 'BPHIGH_CrudePrev', 'BPHIGH_Crude95CI',
       'BPMED_CrudePrev', 'BPMED_Crude95CI', 'CANCER_CrudePrev',
       'CANCER_Crude95CI', 'CASTHMA_CrudePrev', 'CASTHMA_Crude95CI',
       'CHD_CrudePrev', 'CHD_Crude95CI', 'CHECKUP_CrudePrev',
       'CHECKUP_Crude95CI', 'CHOLSCREEN_CrudePrev', 'CHOLSCREEN_Crude95CI',
       'COLON_SCREEN_CrudePrev', 'COLON_SCREEN_Crude95CI', 'COPD_CrudePrev',
       'COPD_Crude95CI', 'COREM_CrudePrev', 'COREM_Crude95CI',
       'COREW_CrudePrev', 'COREW_Crude95CI', 'CSMOKING_CrudePrev',
       'CSMOKING_Crude95CI', 'DENTAL_CrudePrev', 'DENTAL_Crude95CI',
       'DIABETES_CrudePrev', 'DIABETES_Crude95CI', 'HIGHCHOL_CrudePrev',
       'HIGHCHOL_Crude95CI', 'KIDNEY_CrudePrev', 'KIDNEY_Crude95CI',
       'LPA_CrudePrev', 'LPA_

In [12]:
#filter for columns we want
columns_to_keep = [
    "StateAbbr",
    "PlaceName",
    "PlaceFIPS",
    "TractFIPS",
    "Place_TractID",
    "Tract_FIPS_full",
    "Population2010",
    "OBESITY_CrudePrev",
    "DIABETES_CrudePrev",
    "HIGHCHOL_CrudePrev",
    "BPHIGH_CrudePrev",
    "STROKE_CrudePrev",
    "SLEEP_CrudePrev",
    "MHLTH_CrudePrev",
    "CASTHMA_CrudePrev",
    "CHD_CrudePrev",
    "CSMOKING_CrudePrev"
]

# Filter merged dataframe
health_filtered = health_df[columns_to_keep]

## Merge

In [13]:
merged_df = health_filtered.merge(census_df, on="Tract_FIPS_full", how="inner")

In [14]:
merged_df.columns

Index(['StateAbbr', 'PlaceName', 'PlaceFIPS', 'TractFIPS', 'Place_TractID',
       'Tract_FIPS_full', 'Population2010', 'OBESITY_CrudePrev',
       'DIABETES_CrudePrev', 'HIGHCHOL_CrudePrev', 'BPHIGH_CrudePrev',
       'STROKE_CrudePrev', 'SLEEP_CrudePrev', 'MHLTH_CrudePrev',
       'CASTHMA_CrudePrev', 'CHD_CrudePrev', 'CSMOKING_CrudePrev',
       'State_FIPS', 'County_FIPS', 'Tract_FIPS', 'State_Name', 'County_Name',
       'Tract_Name', 'Median_Income', 'Median_Age', 'Poverty_Count',
       'Poverty_Pop', 'HS_Grad_Count', 'Bachelors_Count', 'Education_Pop',
       'Unemployed_Count', 'Unemployed_Pop', 'Race_Total', 'White_Alone',
       'Black_Alone', 'AIAN_Alone', 'Asian_Alone', 'NHPI_Alone', 'Other_Alone',
       'TwoPlus', 'Two_incl_Other', 'Two_excl_Other_or_3plus',
       'Hispanic_Total', 'Hispanic_Any', 'Pct_White_Alone', 'Pct_Black_Alone',
       'Pct_AIAN_Alone', 'Pct_Asian_Alone', 'Pct_NHPI_Alone',
       'Pct_Other_Alone', 'Pct_TwoPlus', 'Majority_Race', 'Pct_Hispanic',
 

In [15]:
merged_df.head()

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Tract_FIPS_full,Population2010,OBESITY_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,...,Pct_Black_Alone,Pct_AIAN_Alone,Pct_Asian_Alone,Pct_NHPI_Alone,Pct_Other_Alone,Pct_TwoPlus,Majority_Race,Pct_Hispanic,Majority_Race_Clean,Pct_Other
0,CA,Los Angeles,644000,6037206032,0644000-06037206032,06037206032,5275,29.5,15.5,38.3,...,0.935126,1.032535,16.637444,0.467563,21.157218,1.149425,White,77.089421,Hispanic,23.806741
1,CA,Pasadena,656000,6037462001,0656000-06037462001,06037462001,3974,32.0,11.6,31.8,...,10.608785,0.000000,9.915232,0.000000,27.254046,3.673260,White,70.459800,Hispanic,30.927305
2,CA,Fullerton,628000,6059011504,0628000-06059011504,06059011504,5473,18.7,4.6,21.1,...,1.626826,0.365206,23.090969,0.000000,8.449535,4.830677,White,30.378486,White,13.645418
3,TX,Longview,4843888,48183000401,4843888-48183000401,48183000401,3371,33.9,9.2,33.9,...,9.589905,2.870662,3.470032,0.000000,0.220820,0.883281,White,7.350158,White,3.974763
4,WI,Madison,5548000,55025002200,5548000-55025002200,55025002200,4254,29.5,8.7,34.6,...,11.116625,0.000000,5.831266,0.000000,0.000000,9.677419,White,8.263027,White,9.677419


In [16]:
merged_df.shape

(27209, 55)

## Cleaning

In [17]:
#rename Crude Columns
merged_df = merged_df.rename(columns={
    "Population2010" : "Health_Pop",
    "OBESITY_CrudePrev": "Obesity_Pct",
    "DIABETES_CrudePrev": "Diabetes_Pct",
    "HIGHCHOL_CrudePrev": "HighChol_Pct",
    "BPHIGH_CrudePrev": "HighBP_Pct",
    "STROKE_CrudePrev": "Stroke_Pct",
    "SLEEP_CrudePrev": "Sleep_Pct",
    "MHLTH_CrudePrev": "MentalHealth_Pct",
    "CASTHMA_CrudePrev": "Asthma_Pct",
    "CHD_CrudePrev": "HeartDisease_Pct",
    "CSMOKING_CrudePrev": "Smoking_Pct"
})

In [18]:
merged_df.head()

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Tract_FIPS_full,Health_Pop,Obesity_Pct,Diabetes_Pct,HighChol_Pct,...,Pct_Black_Alone,Pct_AIAN_Alone,Pct_Asian_Alone,Pct_NHPI_Alone,Pct_Other_Alone,Pct_TwoPlus,Majority_Race,Pct_Hispanic,Majority_Race_Clean,Pct_Other
0,CA,Los Angeles,644000,6037206032,0644000-06037206032,06037206032,5275,29.5,15.5,38.3,...,0.935126,1.032535,16.637444,0.467563,21.157218,1.149425,White,77.089421,Hispanic,23.806741
1,CA,Pasadena,656000,6037462001,0656000-06037462001,06037462001,3974,32.0,11.6,31.8,...,10.608785,0.000000,9.915232,0.000000,27.254046,3.673260,White,70.459800,Hispanic,30.927305
2,CA,Fullerton,628000,6059011504,0628000-06059011504,06059011504,5473,18.7,4.6,21.1,...,1.626826,0.365206,23.090969,0.000000,8.449535,4.830677,White,30.378486,White,13.645418
3,TX,Longview,4843888,48183000401,4843888-48183000401,48183000401,3371,33.9,9.2,33.9,...,9.589905,2.870662,3.470032,0.000000,0.220820,0.883281,White,7.350158,White,3.974763
4,WI,Madison,5548000,55025002200,5548000-55025002200,55025002200,4254,29.5,8.7,34.6,...,11.116625,0.000000,5.831266,0.000000,0.000000,9.677419,White,8.263027,White,9.677419


In [19]:
merged_df.columns

Index(['StateAbbr', 'PlaceName', 'PlaceFIPS', 'TractFIPS', 'Place_TractID',
       'Tract_FIPS_full', 'Health_Pop', 'Obesity_Pct', 'Diabetes_Pct',
       'HighChol_Pct', 'HighBP_Pct', 'Stroke_Pct', 'Sleep_Pct',
       'MentalHealth_Pct', 'Asthma_Pct', 'HeartDisease_Pct', 'Smoking_Pct',
       'State_FIPS', 'County_FIPS', 'Tract_FIPS', 'State_Name', 'County_Name',
       'Tract_Name', 'Median_Income', 'Median_Age', 'Poverty_Count',
       'Poverty_Pop', 'HS_Grad_Count', 'Bachelors_Count', 'Education_Pop',
       'Unemployed_Count', 'Unemployed_Pop', 'Race_Total', 'White_Alone',
       'Black_Alone', 'AIAN_Alone', 'Asian_Alone', 'NHPI_Alone', 'Other_Alone',
       'TwoPlus', 'Two_incl_Other', 'Two_excl_Other_or_3plus',
       'Hispanic_Total', 'Hispanic_Any', 'Pct_White_Alone', 'Pct_Black_Alone',
       'Pct_AIAN_Alone', 'Pct_Asian_Alone', 'Pct_NHPI_Alone',
       'Pct_Other_Alone', 'Pct_TwoPlus', 'Majority_Race', 'Pct_Hispanic',
       'Majority_Race_Clean', 'Pct_Other'],
      dtype='o

In [20]:
merged_df.shape

(27209, 55)

In [21]:
#make new variables for raw count (health data)
#using the Health_Pop (from the health data)
for col in ["Obesity_Pct", "Diabetes_Pct", "HighChol_Pct", "HighBP_Pct",
            "Stroke_Pct", "Sleep_Pct", "MentalHealth_Pct",
            "Asthma_Pct", "HeartDisease_Pct", "Smoking_Pct"]:
    new_col = col.replace("_Pct", "_Count")
    merged_df[new_col] = round((merged_df[col] / 100) * merged_df["Health_Pop"])

In [22]:
#check the raw numbers
merged_df[["Health_Pop", "Obesity_Pct", "Obesity_Count"]].head()

,Health_Pop,Obesity_Pct,Obesity_Count
0,5275,29.5,1556.0
1,3974,32.0,1272.0
2,5473,18.7,1023.0
3,3371,33.9,1143.0
4,4254,29.5,1255.0


In [23]:
#make new variables for percentages (census data)
#using the population from each universe
merged_df["Poverty_Pct"] = (merged_df["Poverty_Count"] / merged_df["Poverty_Pop"]) * 100
merged_df["HS_Grad_Pct"] = (merged_df["HS_Grad_Count"] / merged_df["Education_Pop"]) * 100
merged_df["Bachelors_Pct"] = (merged_df["Bachelors_Count"] / merged_df["Education_Pop"]) * 100
merged_df["Unemployed_Pct"] = (merged_df["Unemployed_Count"] / merged_df["Unemployed_Pop"]) * 100

In [24]:
#check the raw numbers
merged_df[["Education_Pop", "HS_Grad_Pct", "HS_Grad_Count"]].head()

,Education_Pop,HS_Grad_Pct,HS_Grad_Count
0,2920,23.116438,675
1,2419,19.636213,475
2,2481,9.068924,225
3,2261,28.173375,637
4,2902,18.022054,523


In [25]:
merged_df.head()

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Place_TractID,Tract_FIPS_full,Health_Pop,Obesity_Pct,Diabetes_Pct,HighChol_Pct,...,Stroke_Count,Sleep_Count,MentalHealth_Count,Asthma_Count,HeartDisease_Count,Smoking_Count,Poverty_Pct,HS_Grad_Pct,Bachelors_Pct,Unemployed_Pct
0,CA,Los Angeles,644000,6037206032,0644000-06037206032,06037206032,5275,29.5,15.5,38.3,...,211.0,2052.0,791.0,448.0,364.0,823.0,35.834639,23.116438,10.582192,4.823990
1,CA,Pasadena,656000,6037462001,0656000-06037462001,06037462001,3974,32.0,11.6,31.8,...,127.0,1598.0,624.0,374.0,191.0,707.0,24.325713,19.636213,15.171558,5.752670
2,CA,Fullerton,628000,6059011504,0628000-06059011504,06059011504,5473,18.7,4.6,21.1,...,66.0,1828.0,772.0,504.0,115.0,728.0,36.493289,9.068924,35.187424,6.832522
3,TX,Longview,4843888,48183000401,4843888-48183000401,48183000401,3371,33.9,9.2,33.9,...,91.0,1176.0,455.0,314.0,182.0,691.0,14.321767,28.173375,8.889872,4.592980
4,WI,Madison,5548000,55025002200,5548000-55025002200,55025002200,4254,29.5,8.7,34.6,...,123.0,1310.0,502.0,391.0,255.0,706.0,12.195728,18.022054,20.468642,9.886264


In [26]:
merged_df.columns

Index(['StateAbbr', 'PlaceName', 'PlaceFIPS', 'TractFIPS', 'Place_TractID',
       'Tract_FIPS_full', 'Health_Pop', 'Obesity_Pct', 'Diabetes_Pct',
       'HighChol_Pct', 'HighBP_Pct', 'Stroke_Pct', 'Sleep_Pct',
       'MentalHealth_Pct', 'Asthma_Pct', 'HeartDisease_Pct', 'Smoking_Pct',
       'State_FIPS', 'County_FIPS', 'Tract_FIPS', 'State_Name', 'County_Name',
       'Tract_Name', 'Median_Income', 'Median_Age', 'Poverty_Count',
       'Poverty_Pop', 'HS_Grad_Count', 'Bachelors_Count', 'Education_Pop',
       'Unemployed_Count', 'Unemployed_Pop', 'Race_Total', 'White_Alone',
       'Black_Alone', 'AIAN_Alone', 'Asian_Alone', 'NHPI_Alone', 'Other_Alone',
       'TwoPlus', 'Two_incl_Other', 'Two_excl_Other_or_3plus',
       'Hispanic_Total', 'Hispanic_Any', 'Pct_White_Alone', 'Pct_Black_Alone',
       'Pct_AIAN_Alone', 'Pct_Asian_Alone', 'Pct_NHPI_Alone',
       'Pct_Other_Alone', 'Pct_TwoPlus', 'Majority_Race', 'Pct_Hispanic',
       'Majority_Race_Clean', 'Pct_Other', 'Obesity_Count'

In [27]:
#check % of missing data
merged_df.isna().mean().sort_values(ascending=False)

Poverty_Pct         0.002132
Median_Income       0.001323
Bachelors_Pct       0.000294
HS_Grad_Pct         0.000294
Unemployed_Pct      0.000147
                      ...   
Education_Pop       0.000000
Unemployed_Count    0.000000
Unemployed_Pop      0.000000
Race_Total          0.000000
Black_Alone         0.000000
Length: 69, dtype: float64

In [28]:
#shape without NAs (less than 100 rows lost)
merged_df = merged_df.dropna()
merged_df.shape

(27113, 69)

In [29]:
#check % of missing data (should be 0s)
merged_df.isna().mean().sort_values(ascending=False)

StateAbbr           0.0
Pct_White_Alone     0.0
Pct_TwoPlus         0.0
Pct_Other_Alone     0.0
Pct_NHPI_Alone      0.0
                   ... 
Education_Pop       0.0
Unemployed_Count    0.0
Unemployed_Pop      0.0
Race_Total          0.0
Unemployed_Pct      0.0
Length: 69, dtype: float64

In [30]:
#reorder columns for better readability
# List the columns in the order you want

race_cols = [
    'Race_Total','Pct_White_Alone','Pct_Black_Alone','Pct_Asian_Alone',
    'Majority_Race_Clean','Pct_Hispanic','Pct_Other'
]

cols = [
    'StateAbbr', 'PlaceName', 'PlaceFIPS', 'TractFIPS', 'Place_TractID',
    'Tract_FIPS_full', 'Health_Pop',
    'Obesity_Pct', 'Obesity_Count',
    'Diabetes_Pct', 'Diabetes_Count',
    'HighChol_Pct', 'HighChol_Count',
    'HighBP_Pct', 'HighBP_Count',
    'Stroke_Pct', 'Stroke_Count',
    'Sleep_Pct', 'Sleep_Count',
    'MentalHealth_Pct', 'MentalHealth_Count',
    'Asthma_Pct', 'Asthma_Count',
    'HeartDisease_Pct', 'HeartDisease_Count',
    'Smoking_Pct', 'Smoking_Count',
    'State_FIPS', 'County_FIPS', 'Tract_FIPS', 'State_Name', 'County_Name',
    'Tract_Name', 'Median_Income', 'Median_Age',
    'Poverty_Pct', 'Poverty_Count',
    'HS_Grad_Pct', 'HS_Grad_Count',
    'Bachelors_Pct', 'Bachelors_Count',
    'Unemployed_Pct', 'Unemployed_Count'
] + race_cols

# Reorder the DataFrame
merged_df = merged_df[cols]

In [31]:
merged_df.columns

Index(['StateAbbr', 'PlaceName', 'PlaceFIPS', 'TractFIPS', 'Place_TractID',
       'Tract_FIPS_full', 'Health_Pop', 'Obesity_Pct', 'Obesity_Count',
       'Diabetes_Pct', 'Diabetes_Count', 'HighChol_Pct', 'HighChol_Count',
       'HighBP_Pct', 'HighBP_Count', 'Stroke_Pct', 'Stroke_Count', 'Sleep_Pct',
       'Sleep_Count', 'MentalHealth_Pct', 'MentalHealth_Count', 'Asthma_Pct',
       'Asthma_Count', 'HeartDisease_Pct', 'HeartDisease_Count', 'Smoking_Pct',
       'Smoking_Count', 'State_FIPS', 'County_FIPS', 'Tract_FIPS',
       'State_Name', 'County_Name', 'Tract_Name', 'Median_Income',
       'Median_Age', 'Poverty_Pct', 'Poverty_Count', 'HS_Grad_Pct',
       'HS_Grad_Count', 'Bachelors_Pct', 'Bachelors_Count', 'Unemployed_Pct',
       'Unemployed_Count', 'Race_Total', 'Pct_White_Alone', 'Pct_Black_Alone',
       'Pct_Asian_Alone', 'Majority_Race_Clean', 'Pct_Hispanic', 'Pct_Other'],
      dtype='object')

## Save as CSV

In [32]:
#save as csv (won't have to run merging and cleaning again)
merged_df.to_csv("merged_health_census_clean.csv", index=False)

In [33]:
merged_df.shape

(27113, 50)